# Yakemon Battle Simulation & Reinforcement Learning

이 노트북은 다음과 같은 목적으로 사용됩니다:

1. 배틀 시뮬레이션 실행
2. 강화학습 모델 학습
3. 학습된 모델 저장
4. 모델 성능 평가 및 시각화


In [2]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import gym
from IPython.display import clear_output
import time

## 1. 환경 설정 및 시뮬레이션 준비


In [3]:
# 시뮬레이션 파라미터 설정
NUM_EPISODES = 1000
MAX_STEPS = 100
LEARNING_RATE = 0.001
GAMMA = 0.99

# 시각화를 위한 리스트
episode_rewards = []
episode_lengths = []

## 2-1. 배틀 환경 설정


In [ ]:
def setup_battle_environment():
    # 1. 포켓몬 데이터 로드
    from utils.battle_logics.create_battle_pokemon import create_battle_pokemon
    from context.battle_store import store
    from context.duration_store import duration_store
    
    # 2. 랜덤 포켓몬 생성 (예시)
    my_team = []
    enemy_team = []
    for _ in range(3):  # 3마리씩 팀 구성
        my_pokemon = create_battle_pokemon(random_pokemon())  # random_pokemon() 함수는 구현 필요
        enemy_pokemon = create_battle_pokemon(random_pokemon())
        my_team.append(my_pokemon)
        enemy_team.append(enemy_pokemon)
    
    # 3. 배틀 스토어 초기화
    store.set_my_team(my_team)
    store.set_enemy_team(enemy_team)
    store.set_active_my(0)
    store.set_active_enemy(0)
    store.set_turn(1)
    
    # 4. Duration 스토어 초기화
    duration_store.state = {
        "my_effects": [],
        "enemy_effects": [],
        "public_effects": [],
        "my_env_effects": [],
        "enemy_env_effects": []
    }
    
    return store, duration_store

## 2-2. 배틀 시뮬레이션


In [ ]:
from RL import get_state_vector
from utils.battle_logics.battle_sequence import battle_sequence


async def run_battle_simulation():
    store, duration_store = setup_battle_environment()
    
    while True:
        # 1. 현재 상태 벡터 추출
        state = store.get_state()
        state_vector = get_state_vector(
            state["my_team"],
            state["enemy_team"],
            state["active_my"],
            state["active_enemy"],
            state["public_env"],
            state["my_env"],
            state["enemy_env"],
            state["turn"],
            duration_store.state["my_effects"],
            duration_store.state["enemy_effects"]
        )
        
        # 2. 모델로부터 액션 선택 (이 부분은 나중에 구현)
        my_action = model.select_action(state_vector)
        enemy_action = random_action()  # 랜덤 액션 (나중에 다른 모델로 대체 가능)
        
        # 3. 배틀 시퀀스 실행
        await battle_sequence(my_action, enemy_action)
        
        # 4. 게임 종료 체크
        my_team = store.get_team("my")
        enemy_team = store.get_team("enemy")
        
        if all(p.current_hp <= 0 for p in my_team):
            print("상대방 승리!")
            break
        if all(p.current_hp <= 0 for p in enemy_team):
            print("내 승리!")
            break
            
        # 5. 턴 증가
        store.set_turn(state["turn"] + 1)

## 3. 강화학습 모델 정의


In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.softmax(x, dim=-1)

## 4. 모델 학습


In [ ]:
def train_model():
    # TODO: 모델 학습 로직 구현
    pass

## 5. 모델 저장 및 로드


In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    return model

## 6. 성능 평가 및 시각화


In [ ]:
def plot_training_results():
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(episode_rewards)
    plt.title('Episode Rewards')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    
    plt.subplot(1, 2, 2)
    plt.plot(episode_lengths)
    plt.title('Episode Lengths')
    plt.xlabel('Episode')
    plt.ylabel('Length')
    
    plt.tight_layout()
    plt.show()